In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import Dropdown,Accordion,Label,link
%matplotlib inline

In [2]:
df = pd.read_csv("healthcare-dataset-stroke-data.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [4]:
df.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [5]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
df.drop(["id"],axis=1,inplace=True)

In [7]:
sns.countplot(x="stroke",data=clean)

NameError: name 'clean' is not defined

OBSERVATIONS

*  There is a greater chance of Stroke in 
        
        self-employed people.
        In people having BMI around the range of 30
        In people of age more than 70
        In former smokers and almost equally likely in smokers
        In males

In [8]:
plt.figure(figsize=(10,10))
sns.scatterplot(x="bmi",y="avg_glucose_level",data=clean,hue="stroke")
plt.xlabel("BMI")
plt.ylabel("AVG-GLUCOSE-LEVEL")
plt.title("BMI vs AVG-GLUCOSE-LEVEL")

NameError: name 'clean' is not defined

<Figure size 720x720 with 0 Axes>

Average Glucose Level of more than 150 while having a BMI of 20 to 40 might lead to Stroke

In [9]:
yes = clean.where(clean["stroke"]==1).dropna()

NameError: name 'clean' is not defined

# Analysis and Model

## Preprocessing

In [10]:
# most people in the 30 to 40 range, so I am filling values with the mean
df["bmi"].replace({np.nan:np.mean(df["bmi"])},inplace=True)
df.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [11]:
# One-Hot Encoding
df = pd.get_dummies(df)
reorder = list(df.columns)
reorder.remove("stroke")
reorder.append("stroke")
df = df[reorder]
df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,...,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,67.0,0,1,228.69,36.600000,0,1,0,0,1,...,1,0,0,0,1,0,1,0,0,1
1,61.0,0,0,202.21,28.893237,1,0,0,0,1,...,0,1,0,1,0,0,0,1,0,1
2,80.0,0,1,105.92,32.500000,0,1,0,0,1,...,1,0,0,1,0,0,0,1,0,1
3,49.0,0,0,171.23,34.400000,1,0,0,0,1,...,1,0,0,0,1,0,0,0,1,1
4,79.0,1,0,174.12,24.000000,1,0,0,0,1,...,0,1,0,1,0,0,0,1,0,1


In [12]:
# Handling low distribution
from sklearn.utils import resample
df_majority = df[df.stroke == 0]
df_minority = df[df.stroke == 1]

df_upsampled = resample(df_minority,
                       replace=True,
                       n_samples=4861,
                       random_state=0)

df_upsampled = pd.concat([df_majority,df_upsampled])

df_upsampled.stroke.value_counts()

1    4861
0    4861
Name: stroke, dtype: int64

In [13]:
# Splitting the Data
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

from sklearn.model_selection import train_test_split as tts

x_train,x_val,y_train,y_val = tts(X,y,test_size=0.4,random_state=0)
x_val,x_test,y_val,y_test = tts(x_val,y_val,test_size=0.5,random_state=0)

print([s.shape for s in [x_train,x_val,x_test,y_train,y_val,y_test]])

[(3066, 21), (1022, 21), (1022, 21), (3066,), (1022,), (1022,)]


In [14]:
# Classification using Gradient_Boosting Algorithm
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=1, random_state=0).fit(x_train, y_train)
print(model.score(x_test,y_test)*100,'%')

95.10763209393346 %


In [15]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
target_names = ["NO","YES"]
print(classification_report(y_test, y_pred, target_names=target_names,zero_division=1))

              precision    recall  f1-score   support

          NO       0.95      1.00      0.97       972
         YES       1.00      0.00      0.00        50

    accuracy                           0.95      1022
   macro avg       0.98      0.50      0.49      1022
weighted avg       0.95      0.95      0.93      1022

